In [1]:
%pip install yt-dlp

In [2]:

import os
import tensorflow as tf
import numpy as np
import yt_dlp as yt_dlp
import cv2
import json
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, TimeDistributed, Conv2D, Flatten, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input


from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/Colab Notebooks/MS-ASL/MSASL_train.json', 'r') as file:
    train_data = json.load(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def download_clip(url, output_filename):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]',  # This ensures mp4 format is downloaded
        'outtmpl': output_filename,
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [4]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)

    cap.release()
    return frames


In [ ]:
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_dataset/"
SEQUENCE_LENGTH = 15
private_videos = []  # List to store details of private videos
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Modify preprocess_data to save frames to disk and process all data entries
def preprocess_and_save_all_data(data, data_path=DATA_PATH):
    label_encoder = LabelEncoder()
    labels = [item['text'] for item in data]
    encoded_labels = label_encoder.fit_transform(labels)

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    for idx, item in enumerate(data):
        video_filename = f"video_{idx}.mp4"
        try:
            download_clip(item['url'], video_filename)
            print(f"Successfully downloaded video {idx} from URL: {item['url']}")

            frames = extract_frames(video_filename)
            print(f"Extracted {len(frames)} frames from video {idx}.")
            frames = [cv2.resize(frame, (IMG_HEIGHT, IMG_WIDTH)) for frame in frames]
            frames = [preprocess_input(frame) for frame in frames]

            # Save frames to disk
            class_dir = os.path.join(data_path, item['text'])
            video_dir = os.path.join(class_dir, f"video_{idx}")
            if not os.path.exists(video_dir):
                os.makedirs(video_dir)
            for frame_idx, frame in enumerate(frames):
                frame_path = os.path.join(video_dir, f"frame_{frame_idx}.jpg")
                cv2.imwrite(frame_path, frame)
                #print("Frame saved")

        except Exception as e:
            if "Private video" in str(e):
                private_videos.append(item['url'])  # Log the URL of the private video
            continue  # Skip to the next video

preprocess_and_save_all_data(train_data)




[youtube] Extracting URL: https://www.youtube.com/watch?v=C37R_Ix8-qs
[youtube] C37R_Ix8-qs: Downloading webpage
[youtube] C37R_Ix8-qs: Downloading ios player API JSON
[youtube] C37R_Ix8-qs: Downloading android player API JSON
[youtube] C37R_Ix8-qs: Downloading m3u8 information
[info] C37R_Ix8-qs: Downloading 1 format(s): 137
[download] video_0.mp4 has already been downloaded
[download] 100% of  417.79KiB
[VideoConvertor] Not converting media file "video_0.mp4"; already is in target format mp4
Successfully downloaded video 0 from URL: https://www.youtube.com/watch?v=C37R_Ix8-qs
Extracted 85 frames from video 0.
[youtube] Extracting URL: https://www.youtube.com/watch?v=PIsUJl8BN_I
[youtube] PIsUJl8BN_I: Downloading webpage
[youtube] PIsUJl8BN_I: Downloading ios player API JSON
[youtube] PIsUJl8BN_I: Downloading android player API JSON
[youtube] PIsUJl8BN_I: Downloading m3u8 information
[info] PIsUJl8BN_I: Downloading 1 format(s): 135
[download] video_1.mp4 has already been downloaded
[d

[youtube] Extracting URL: http://www.youtube.com/watch?v=9FdHlMOnVjg
[youtube] 9FdHlMOnVjg: Downloading webpage
[youtube] 9FdHlMOnVjg: Downloading ios player API JSON
[youtube] 9FdHlMOnVjg: Downloading android player API JSON


ERROR: [youtube] 9FdHlMOnVjg: Private video. Sign in if you've been granted access to this video


[youtube] Extracting URL: https://www.youtube.com/watch?v=J7tP98oDxqE
[youtube] J7tP98oDxqE: Downloading webpage
[youtube] J7tP98oDxqE: Downloading ios player API JSON
[youtube] J7tP98oDxqE: Downloading android player API JSON
[youtube] J7tP98oDxqE: Downloading m3u8 information
[youtube] J7tP98oDxqE: Downloading MPD manifest
[info] J7tP98oDxqE: Downloading 1 format(s): 135
[download] video_3.mp4 has already been downloaded
[download] 100% of   81.88KiB
[VideoConvertor] Not converting media file "video_3.mp4"; already is in target format mp4
Successfully downloaded video 3 from URL: https://www.youtube.com/watch?v=J7tP98oDxqE
Extracted 68 frames from video 3.
[generic] Extracting URL: www.youtube.com/watch?v=N2mG9ZKjrGA


[youtube] Extracting URL: http://www.youtube.com/watch?v=N2mG9ZKjrGA
[youtube] N2mG9ZKjrGA: Downloading webpage
[youtube] N2mG9ZKjrGA: Downloading ios player API JSON
[youtube] N2mG9ZKjrGA: Downloading android player API JSON
[youtube] N2mG9ZKjrGA: Downloading m3u8 information
[info] N2mG9ZKjrGA: Downloading 1 format(s): 137
[download] video_4.mp4 has already been downloaded
[download] 100% of  400.60KiB
[VideoConvertor] Not converting media file "video_4.mp4"; already is in target format mp4
Successfully downloaded video 4 from URL: www.youtube.com/watch?v=N2mG9ZKjrGA
Extracted 77 frames from video 4.
[youtube] Extracting URL: https://www.youtube.com/watch?v=1AyT77LqJzQ
[youtube] 1AyT77LqJzQ: Downloading webpage
[youtube] 1AyT77LqJzQ: Downloading ios player API JSON
[youtube] 1AyT77LqJzQ: Downloading android player API JSON


...

ERROR: [youtube] 7y5Ye-2-ZBs: Private video. Sign in if you've been granted access to this video


[youtube] Extracting URL: https://www.youtube.com/watch?v=I_QDeKcyvns
[youtube] I_QDeKcyvns: Downloading webpage
[youtube] I_QDeKcyvns: Downloading ios player API JSON
[youtube] I_QDeKcyvns: Downloading android player API JSON
[youtube] I_QDeKcyvns: Downloading m3u8 information
[youtube] I_QDeKcyvns: Downloading MPD manifest
[info] I_QDeKcyvns: Downloading 1 format(s): 270
[download] video_16.mp4 has already been downloaded
[download] 100% of  730.42KiB
[VideoConvertor] Not converting media file "video_16.mp4"; already is in target format mp4
Successfully downloaded video 16 from URL: https://www.youtube.com/watch?v=I_QDeKcyvns
Extracted 115 frames from video 16.
[youtube] Extracting URL: https://www.youtube.com/watch?v=WeAFuzYTdtU
[youtube] WeAFuzYTdtU: Downloading webpage
[youtube] WeAFuzYTdtU: Downloading ios player API JSON
[youtube] WeAFuzYTdtU: Downloading android player API JSON
[youtube] WeAFuzYTdtU: Downloading m3u8 information
[info] WeAFuzYTdtU: Downloading 1 format(s): 606


[youtube] Extracting URL: http://www.youtube.com/watch?v=rYW7VFPi9_w
[youtube] rYW7VFPi9_w: Downloading webpage
[youtube] rYW7VFPi9_w: Downloading ios player API JSON
[youtube] rYW7VFPi9_w: Downloading android player API JSON


ERROR: [youtube] rYW7VFPi9_w: Private video. Sign in if you've been granted access to this video


[youtube] Extracting URL: https://www.youtube.com/watch?v=bUezx9Zp16E
[youtube] bUezx9Zp16E: Downloading webpage
[youtube] bUezx9Zp16E: Downloading ios player API JSON
[youtube] bUezx9Zp16E: Downloading android player API JSON
[youtube] bUezx9Zp16E: Downloading m3u8 information
[youtube] bUezx9Zp16E: Downloading MPD manifest
[info] bUezx9Zp16E: Downloading 1 format(s): 311
[download] video_30.mp4 has already been downloaded
[download] 100% of  555.19KiB
[VideoConvertor] Not converting media file "video_30.mp4"; already is in target format mp4
Successfully downloaded video 30 from URL: https://www.youtube.com/watch?v=bUezx9Zp16E
Extracted 144 frames from video 30.
[youtube] Extracting URL: https://www.youtube.com/watch?v=jQb9NL9_S6U
[youtube] jQb9NL9_S6U: Downloading webpage
[youtube] jQb9NL9_S6U: Downloading ios player API JSON
[youtube] jQb9NL9_S6U: Downloading android player API JSON
[youtube] jQb9NL9_S6U: Downloading m3u8 information
[youtube] jQb9NL9_S6U: Downloading MPD manifest
[

In [ ]:


def load_data(data_path):
    X, y = [], []
    classes = sorted([d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))])
    label_encoder = LabelEncoder()
    encoded_classes = label_encoder.fit_transform(classes)

    for idx, class_name in enumerate(classes):
        class_path = os.path.join(data_path, class_name)
        videos = os.listdir(class_path)

        for video_name in videos:
            video_path = os.path.join(class_path, video_name)
            frames = [os.path.join(video_path, frame) for frame in os.listdir(video_path) if frame.endswith('.jpg')]
            frames = sorted(frames, key=lambda x: int(x.split('_')[-1].split('.')[0]))

            # Load and preprocess frames
            loaded_frames = [image.load_img(frame, target_size=(IMG_HEIGHT, IMG_WIDTH)) for frame in frames]
            loaded_frames = [image.img_to_array(frame_img) for frame_img in loaded_frames]
            loaded_frames = [preprocess_input(frame_arr) for frame_arr in loaded_frames]

            # Create sequences
            for i in range(len(loaded_frames) - SEQUENCE_LENGTH + 1):
                X.append(loaded_frames[i:i+SEQUENCE_LENGTH])
                y.append(encoded_classes[idx])

    X = np.array(X)
    y = to_categorical(y, num_classes=len(classes))

    return X, y, label_encoder


[youtube] Extracting URL: https://www.youtube.com/watch?v=C37R_Ix8-qs
[youtube] C37R_Ix8-qs: Downloading webpage
[youtube] C37R_Ix8-qs: Downloading ios player API JSON
[youtube] C37R_Ix8-qs: Downloading android player API JSON
[youtube] C37R_Ix8-qs: Downloading m3u8 information
[info] C37R_Ix8-qs: Downloading 1 format(s): 137
[download] video_0.mp4 has already been downloaded
[download] 100% of  417.79KiB
[VideoConvertor] Not converting media file "video_0.mp4"; already is in target format mp4
Successfully downloaded video 0 from URL: https://www.youtube.com/watch?v=C37R_Ix8-qs
Extracted 85 frames from video 0.
[youtube] Extracting URL: https://www.youtube.com/watch?v=PIsUJl8BN_I
[youtube] PIsUJl8BN_I: Downloading webpage
[youtube] PIsUJl8BN_I: Downloading ios player API JSON
[youtube] PIsUJl8BN_I: Downloading android player API JSON
[youtube] PIsUJl8BN_I: Downloading m3u8 information
[info] PIsUJl8BN_I: Downloading 1 format(s): 135
[download] video_1.mp4 has already been downloaded
[d

[youtube] Extracting URL: http://www.youtube.com/watch?v=9FdHlMOnVjg
[youtube] 9FdHlMOnVjg: Downloading webpage
[youtube] 9FdHlMOnVjg: Downloading ios player API JSON
[youtube] 9FdHlMOnVjg: Downloading android player API JSON


ERROR: [youtube] 9FdHlMOnVjg: Private video. Sign in if you've been granted access to this video


[youtube] Extracting URL: https://www.youtube.com/watch?v=J7tP98oDxqE
[youtube] J7tP98oDxqE: Downloading webpage
[youtube] J7tP98oDxqE: Downloading ios player API JSON
[youtube] J7tP98oDxqE: Downloading android player API JSON
[youtube] J7tP98oDxqE: Downloading m3u8 information
[youtube] J7tP98oDxqE: Downloading MPD manifest
[info] J7tP98oDxqE: Downloading 1 format(s): 135
[download] video_3.mp4 has already been downloaded
[download] 100% of   81.88KiB
[VideoConvertor] Not converting media file "video_3.mp4"; already is in target format mp4
Successfully downloaded video 3 from URL: https://www.youtube.com/watch?v=J7tP98oDxqE
Extracted 68 frames from video 3.
[generic] Extracting URL: www.youtube.com/watch?v=N2mG9ZKjrGA


[youtube] Extracting URL: http://www.youtube.com/watch?v=N2mG9ZKjrGA
[youtube] N2mG9ZKjrGA: Downloading webpage
[youtube] N2mG9ZKjrGA: Downloading ios player API JSON
[youtube] N2mG9ZKjrGA: Downloading android player API JSON
[youtube] N2mG9ZKjrGA: Downloading m3u8 information
[info] N2mG9ZKjrGA: Downloading 1 format(s): 137
[download] video_4.mp4 has already been downloaded
[download] 100% of  400.60KiB
[VideoConvertor] Not converting media file "video_4.mp4"; already is in target format mp4
Successfully downloaded video 4 from URL: www.youtube.com/watch?v=N2mG9ZKjrGA
Extracted 77 frames from video 4.
[youtube] Extracting URL: https://www.youtube.com/watch?v=1AyT77LqJzQ
[youtube] 1AyT77LqJzQ: Downloading webpage
[youtube] 1AyT77LqJzQ: Downloading ios player API JSON
[youtube] 1AyT77LqJzQ: Downloading android player API JSON


...

[generic] Extracting URL: www.youtube.com/watch?v=7y5Ye-2-ZBs


[youtube] Extracting URL: http://www.youtube.com/watch?v=7y5Ye-2-ZBs
[youtube] 7y5Ye-2-ZBs: Downloading webpage
[youtube] 7y5Ye-2-ZBs: Downloading ios player API JSON
[youtube] 7y5Ye-2-ZBs: Downloading android player API JSON


ERROR: [youtube] 7y5Ye-2-ZBs: Private video. Sign in if you've been granted access to this video


[youtube] Extracting URL: https://www.youtube.com/watch?v=I_QDeKcyvns
[youtube] I_QDeKcyvns: Downloading webpage
[youtube] I_QDeKcyvns: Downloading ios player API JSON
[youtube] I_QDeKcyvns: Downloading android player API JSON
[youtube] I_QDeKcyvns: Downloading m3u8 information
[youtube] I_QDeKcyvns: Downloading MPD manifest
[info] I_QDeKcyvns: Downloading 1 format(s): 270
[download] video_16.mp4 has already been downloaded
[download] 100% of  730.42KiB
[VideoConvertor] Not converting media file "video_16.mp4"; already is in target format mp4
Successfully downloaded video 16 from URL: https://www.youtube.com/watch?v=I_QDeKcyvns
Extracted 115 frames from video 16.
[youtube] Extracting URL: https://www.youtube.com/watch?v=WeAFuzYTdtU
[youtube] WeAFuzYTdtU: Downloading webpage
[youtube] WeAFuzYTdtU: Downloading ios player API JSON
[youtube] WeAFuzYTdtU: Downloading android player API JSON
[youtube] WeAFuzYTdtU: Downloading m3u8 information
[info] WeAFuzYTdtU: Downloading 1 format(s): 606


KeyboardInterrupt: ignored

In [ ]:


NUM_CLASSES = len(label_encoder.classes_)

def create_model():
    vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    for layer in vgg_model.layers:
        layer.trainable = False

    input_layer = Input(shape=(SEQUENCE_LENGTH, IMG_HEIGHT, IMG_WIDTH, 3))
    x = TimeDistributed(vgg_model)(input_layer)
    x = TimeDistributed(Flatten())(x)
    x = LSTM(256, activation='relu', return_sequences=False)(x)
    output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_model()

BATCH_SIZE = 16
EPOCHS = 20

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=BATCH_SIZE, epochs=EPOCHS)


58889256/58889256 [==============================] - 0s 0us/step


ValueError: ignored